In [112]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
# from src.metrics import precision_at_k, recall_at_k

С самим зада

In [113]:
data = pd.read_csv('data/transaction_data.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [147]:
item_features = pd.read_csv('data/product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(10)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,
2,26093,69,PASTRY,Private,BREAD,BREAD:ITALIAN/FRENCH,
3,26190,69,GROCERY,Private,FRUIT - SHELF STABLE,APPLE SAUCE,50 OZ
4,26355,69,GROCERY,Private,COOKIES/CONES,SPECIALTY COOKIES,14 OZ
5,26426,69,GROCERY,Private,SPICES & EXTRACTS,SPICES & SEASONINGS,2.5 OZ
6,26540,69,GROCERY,Private,COOKIES/CONES,TRAY PACK/CHOC CHIP COOKIES,16 OZ
7,26601,69,DRUG GM,Private,VITAMINS,VITAMIN - MINERALS,300CT(1)
8,26636,69,PASTRY,Private,BREAKFAST SWEETS,SW GDS: SW ROLLS/DAN,
9,26691,16,GROCERY,Private,PNT BTR/JELLY/JAMS,HONEY,12 OZ


In [55]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[879517, 934369, 1115576, 1124029, 5572301, 65..."
1,3,"[823704, 834117, 840244, 913785, 917816, 93870..."


In [117]:
data_train.loc[data_train['quantity'] == 0]

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
97,744,26985165432,1,5978648,0,0.0,31582,0.0,1119,1,0.0,0.0
128,1287,26985336468,1,5978648,0,0.0,304,0.0,1351,1,0.0,0.0
249,2305,26996870743,2,5978656,0,0.0,414,0.0,1300,1,-1.0,0.0
293,271,26997082949,2,5978656,0,0.0,329,0.0,1719,1,-2.0,0.0
694,315,27008952267,3,957951,0,0.0,327,0.0,1707,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2484136,2205,41931621283,684,15596834,0,0.0,443,0.0,1401,98,0.0,0.0
2484379,889,41931660766,684,5978648,0,0.0,433,0.0,1140,98,0.0,0.0
2484533,227,41931670834,684,6423848,0,0.0,298,0.0,1200,98,0.0,0.0
2484643,919,41931678307,684,17249071,0,0.0,345,0.0,1848,98,0.0,0.0


In [ ]:
def prefilter_items(data):
    # Уберем самые популярные товары (их и так купят)
    popularity = data_train.groupby('item_id')['user_id'].nunique().reset_index() / data_train['user_id'].nunique()
    popularity.rename(columns={'user_id': 'share_unique_users'}, inplace=True)
    
    top_popular = popularity[popularity['share_unique_users'] > 0.5].item_id.tolist()
    data = data[~data['item_id'].isin(top_popular)]
    
    # Уберем самые НЕ популярные товары (их и так НЕ купят)
    top_notpopular = popularity[popularity['share_unique_users'] < 0.01].item_id.tolist()
    data = data[~data['item_id'].isin(top_notpopular)]
    
    # Уберем товары, которые не продавались за последние 12 месяцев
    non_sale = data_train[['item_id','quantity', 'week_no']]
    non_sale = non_sale[non_sale['quantity'] == 0]
    non_sale12 = non_sale[non_sale['week_no'] > 86].item_id.tolist()
    data = data[~data['item_id'].isin(non_sale12)]
    
    # Уберем не интересные для рекоммендаций категории (department)
    non_interest = item_features[['item_id','department']]
    non_interest = non_interest.query("department in ['KIOSK-GAS', 'SEAFOOD-PCKGD', 'MISC SALES TRAN', 'TRAVEL & LEISUR','MISC. TRANS.','FROZEN GROCERY', 'COUP/STR & MFG', 'ELECT &PLUMBING', 'TOYS', 'DELI/SNACK BAR', 'PORK', 'AUTOMOTIVE']").item_id.tolist()
    data = data[~data['item_id'].isin(non_interest)]
    
    # Уберем слишком дешевые товары (на них не заработаем). 1 покупка из рассылок стоит 60 руб. 
    cheap = data_train.groupby('item_id')['sales_value'].min().reset_index()
    cheap.rename(columns={'sales_value': 'min_sales_value'}, inplace=True)
    cheap_final = cheap[cheap['min_sales_value'] < 5].item_id.tolist()
    data = data[~data['item_id'].isin(cheap_final)]
    
    # Уберем слишком дорогие товарыs
    cheap = data_train.groupby('item_id')['sales_value'].max().reset_index()
    cheap.rename(columns={'sales_value': 'min_sales_value'}, inplace=True)
    cheap_final = cheap[cheap['min_sales_value'] > 500].item_id.tolist()
    data = data[~data['item_id'].isin(cheap_final)]
    
    # ...
    
def postfilter_items(user_id, recommednations):
    pass